# Introduction to Bruges

This notebook accompanies [a blog post on agilegeoscience.com](http://www.agilegeoscience.com/blog/). 

You need to install Bruges to run this notebook. You can clone the repository on GitHub or if you have pip installed, in a terminal you can do,

    pip install bruges

This notebook also requires Welly, which you can install like so:

    pip install welly

In [ ]:
import bruges as b
b.__version__

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

<hr />

## AVO calculations

### Offset dependent reflectivity

|  | Vp [m/s] | Vs [m/s] | Density [kg/m3] |  
|-------------|----------|----------|----------|
| upper layer | 3300 | 1500 | 2400 |
| lower layer | 3050 | 1400 | 2075 |

In [ ]:
# Upper layer rock properties
vp1 = 3300.0
vs1 = 1500.0
rho1 = 2400.0

# Lower layer rock properties
vp2 = 3050.0
vs2 = 1400.0
rho2 = 2075.0

In [ ]:
rc_0 = b.reflection.akirichards(vp1, vs1, rho1, vp2, vs2, rho2, theta1=0)
rc_0

It might be more convenient to use list unpacking. That way we can pass the arguments as two lists, or 'rocks':

In [ ]:
rock1 = [vp1, vs1, rho1]
rock2 = [vp2, vs2, rho2]

In [ ]:
rc_0 = b.reflection.akirichards(*rock1, *rock2, theta1=0)
rc_0

In [ ]:
rc_30 = b.reflection.akirichards(*rock1, *rock2, theta1=30)
rc_30

We can also get the individual terms of Shuey's linear approximation:

In [ ]:
rc_terms = b.reflection.shuey3(*rock1, *rock2, theta1=30, terms=True)
rc_terms

The second term is the product of gradient and $sin^2 \theta$. You can get at the gradient directly with an option:

In [ ]:
intercept, gradient = b.reflection.shuey(*rock1, *rock2, theta1=30, return_gradient=True)
intercept, gradient

You can equally well pass in a list or array of angles,

In [ ]:
theta_list = [0, 10, 20, 30]
rc_list = b.reflection.akirichards(*rock1, *rock2, theta1=theta_list)
rc_list

Create an array of angles from 0 to 70, incremented by 1,

In [ ]:
theta_range = np.arange(0, 70)

In [ ]:
rc_range = b.reflection.akirichards(*rock1, *rock2, theta1=theta_range)

Compare the two-term Aki-Richards approximation with the full Zoeprittz equation for a interface between two rocks:

In [ ]:
rc_z = b.reflection.zoeppritz_rpp(*rock1, *rock2, theta1=theta_range)

Put all this data on an AVO plot:

In [ ]:
style = {'color': 'blue',
         'fontsize': 10,
         'ha':'left',
         'va':'top',}

fig = plt.figure(figsize=(12,5))

# AVO plot
ax1 = fig.add_subplot(121)
ax1.plot(theta_range, rc_z, 'k', lw=3, alpha=0.25, label='Zoeppritz')
ax1.plot(theta_range, rc_range, 'k.', lw=3, alpha=0.5, label='Aki-Richards')

# We'll also add the four angles...
ax1.plot(theta_list, rc_list, 'bo', ms=10, alpha = 0.5)

# Putting some annotations on the plot.
for theta, rc in zip(theta_list, rc_list):
    ax1.text(theta, rc-0.004, '{:.3f}'.format(rc), **style)

ax1.legend()
ax1.set_ylim((-0.15, -0.05))
ax1.set_xlabel('Angle (degrees)')
ax1.set_ylabel('Amplitude')
ax1.grid()

# Intercept-Gradient crossplot.
ax2 = fig.add_subplot(122)
ax2.plot(intercept, gradient, 'ko', ms=10, alpha = 0.5)

# Put spines for x and y axis.
ax2.axvline(0, color='k')
ax2.axhline(0, color='k')

# Set square axes limits.
mx = 0.25
ax2.set_xlim((-mx, mx))
ax2.set_ylim((-mx, mx))

# Label the axes and add gridlines.
ax2.set_xlabel('Intercept')
ax2.set_ylabel('Gradient')
ax2.grid()

plt.show()

<hr />

## Elastic moduli calculations

Say I want to compute the Lamé parameters &#955; and &#181;, from V<sub>P</sub>, V<sub>S</sub>, and Density. As long as my inputs are in SI units, I can insert these values directly:

|  | Vp [m/s] | Vs [m/s] | Density [kg/m3] |
|-------------|----------|----------|----------|
| upper layer | 3300 | 1500 | 2400 |
| lower layer | 3050 | 1400 | 2075 |

In [ ]:
# Upper layer only
b.rockphysics.lam(*rock1), b.rockphysics.mu(*rock1)

We can print all the values in terms of GPa ($\times 10^9$ Pa)

In [ ]:
print('Upper layer')
print('lambda [GPa]: {:.2f}'.format(b.rockphysics.lam(*rock1) / 1e9))
print('    mu [GPa]: {:.2f}'.format(b.rockphysics.mu(*rock1) / 1e9))
print('\nLower layer')
print('lambda [GPa]: {:.2f}'.format(b.rockphysics.lam(*rock2) / 1e9))
print('    mu [GPa]: {:.2f}'.format(b.rockphysics.mu(*rock2) / 1e9))

<hr />

## Rock property analysis and crossplotting

### Load some data

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import bruges as b
%matplotlib inline

import welly
welly.__version__

In [ ]:
w = welly.Well.from_las('../data/L-30.las')

In [ ]:
p = welly.Project([w])
from IPython.display import HTML
HTML(p.curve_table_html())

In [ ]:
w.plot()

In [ ]:
w.data['RHOB']

Turn things into Python variables for convenience.

In [ ]:
dt = w.data['DT']
rhob = w.data['RHOB']

The log has a lot of NaNs. We note that it's shorter than DT. We can try to mitigate this a bit like:

In [ ]:
rhob, dt = b.util.top_and_tail(rhob, dt)

In [ ]:
z = dt.basis * 0.3048

Convert everything to SI units:

In [ ]:
vp =  1e6 / (3.28084 * dt)
vp.units = 'm/s'
vs = (vp - 1360) / 1.16  # Castanga model
rhob *= 1000
rhob.units = 'kg/m3'

### Now the rock properties

In [ ]:
lm = b.rockphysics.lam(vp, vs, rhob)
mu = b.rockphysics.mu(vp, vs, rhob)

Create a crossplot:

In [ ]:
plt.figure(figsize=(7,6))

plt.scatter(lm*rhob, mu*rhob,
            s=30,  # marker size
            color=z, cmap="gist_earth_r",
            edgecolor='none',
            alpha = 0.1)

# Give the plot a colorbar.
cb = plt.colorbar()
cb.ax.invert_yaxis()
cb.set_label("Depth [m]")

# Give the plot some annotation.
plt.xlabel(r'$\lambda \rho$', size=18)
plt.ylabel(r'$\mu \rho$', size=18)
plt.grid()

<hr />

## Backus averaging

In [ ]:
lb = 60       # Backus averaging length in metres.
dz = 0.1524   # Sample interval of the log in metres.

In [ ]:
vp0, vs0, rhob_filt = b.rockphysics.backus(vp, vs, rhob, lb, dz)

In [ ]:
fs = 8  # control the fontsize

fig = plt.figure(figsize=(3,12))

ax1 = plt.subplot(111)
ax1.plot(vp, z, 'k', alpha=0.25, label='Vp')
ax1.plot(vp0, z, 'b', label='Backus average')

# Do some plot annotations
ax1.set_title('P-velocity')
ax1.set_ylabel(r'depth $m$', size=12)
ax1.invert_yaxis()
#ax1.set_xlim((4000,6200))
ax1.set_xlabel(r'Vp $m/s$')
ax1.grid()
ax1.legend()
labels = ax1.get_xticklabels() 
for label in labels: 
    label.set_rotation(90) 
    label.set_fontsize(fs)

plt.show()

<hr />
<img src="https://avatars1.githubusercontent.com/u/1692321?v=3&s=200" style="float:right;" width="40px" /><p style="color:gray; float:right;">© 2015 <a href="http://www.agilegeoscience.com/">Agile Geoscience</a> — <a href="https://creativecommons.org/licenses/by/4.0/">CC-BY</a> — Have fun!&nbsp;&nbsp;&nbsp;</p>